# Test Semantic Search on Labeled Set

In [ ]:
import pandas as pd
from semantic_search.data import build_corpus
from semantic_search.local import LocalKnowledgeBase

## Configs

In [14]:
LEVEL   : int = 2
MODEL_ID: str = "paraphrase-multilingual-MiniLM-L12-v2"

---

## Data

In [15]:
test_df = pd.read_csv("classification/ateco_2025/ateco_2025_index.csv")
ateco_df = pd.read_csv(f"classification/ateco_2025/ateco_2025_level_{LEVEL}.csv")

descriptor_template = """# {title}
{description}"""

descriptors = [descriptor_template.format(title=t, description=d) for t, d in zip(ateco_df["title"], ateco_df["description"])]

y_texts, y_true = test_df["COMBO_PARLATA_IT"].tolist(), test_df["ATECO2025"].tolist()

Create the knowledge base.

In [16]:
corpus = build_corpus(
    texts=descriptors,
    ids=ateco_df.index,
    metadata=[{"code": c} for c in ateco_df["code"]],
)

base = LocalKnowledgeBase(
    corpus,
    model_id=MODEL_ID,
    batch_size=64
)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  9.87it/s]


## Semantic Search

In [18]:
TOP_K = 5

results = base.search(y_texts, top_k=TOP_K)

correct = 0
for result, yt in zip(results, y_true):
    codes = [r.metadata["code"] for r in result]
    scores = [r.score for r in result]

    if any(str(code) in yt for code in codes):
        correct += 1

print(f"Accuracy (Top {TOP_K}): {correct / len(y_true):.2%} ({correct}/{len(y_true)})")

Batches:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0/119 [00:00<?, ?it/s]

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:01<00:00, 69.12it/s]


Accuracy (Top 5): 71.08% (5412/7614)
